<a href="https://colab.research.google.com/github/shakasom/geocoding/blob/master/ReverseGeocoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Install GDAL and Geopandas
!apt install gdal-bin python-gdal python3-gdal --quiet
!apt install python3-rtree --quiet
!pip install git+git://github.com/geopandas/geopandas.git --quiet
!pip install descartes --quiet
!pip install geopy
!pip install plotly_express
!pip install ipython-autotime


Reading package lists...
Building dependency tree...
Reading state information...
gdal-bin is already the newest version (2.2.3+dfsg-2).
python-gdal is already the newest version (2.2.3+dfsg-2).
python3-gdal is already the newest version (2.2.3+dfsg-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.
Reading package lists...
Building dependency tree...
Reading state information...
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
  python3-pkg-resources
Suggested packages:
  python3-setuptools
The following NEW packages will be installed:
  libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
  python3-pkg-resources python3-rtree
0 upg

* Restart the run time. tqdm does not work with recent python so we downgrade it to get it working

In [0]:
!pip install tqdm==4.36.0

     |████████████████████████████████| 61kB 6.1MB/s 
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [0]:
%load_ext autotime
import pandas as pd

import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

import matplotlib.pyplot as plt
import plotly_express as px
import tqdm
from tqdm import tqdm
from tqdm._tqdm_notebook import tqdm_notebook

## Reverse Geocoding Single Example

In [0]:
locator = Nominatim(user_agent="myGeocoder")
coordinates = "53.480837, -2.244914"
location = locator.reverse(coordinates)

time: 784 ms


In [0]:
location.raw

{'address': {'building': 'Eagle Insurance Buildings',
  'city': 'Manchester',
  'country': 'United Kingdom',
  'country_code': 'gb',
  'county': 'Greater Manchester',
  'house_number': '68',
  'postcode': 'M2 4JG',
  'road': 'Cross Street',
  'state': 'England',
  'state_district': 'North West England',
  'suburb': 'City Centre'},
 'boundingbox': ['53.480856', '53.4810634', '-2.2451761', '-2.2449576'],
 'display_name': 'Eagle Insurance Buildings, 68, Cross Street, City Centre, Manchester, Greater Manchester, North West England, England, M2 4JG, United Kingdom',
 'lat': '53.4809597',
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'lon': '-2.24506682746292',
 'osm_id': 37139875,
 'osm_type': 'way',
 'place_id': 86197794}

time: 8.84 ms


In [0]:
print(location.address)

Eagle Insurance Buildings, 68, Cross Street, City Centre, Manchester, Greater Manchester, North West England, England, M2 4JG, United Kingdom
time: 1.69 ms


In [0]:
location.raw["display_name"]

'Eagle Insurance Buildings, 68, Cross Street, City Centre, Manchester, Greater Manchester, North West England, England, M2 4JG, United Kingdom'

time: 4.28 ms


## Reverse Geocoding with Pandas

In [0]:
url = "https://www.dropbox.com/s/15gisj8hx218rn1/street-pole-sample.csv?dl=1"
cols = ["X", "Y", "POLE_NUM","TYPE","HEIGHT","POLE_DATE","OWNER"]
df = pd.read_csv(url, usecols=cols)
df.head()

,X,Y,POLE_NUM,TYPE,HEIGHT,POLE_DATE,OWNER
0,-75.170097,39.942766,214423,WP,NaN,1997-06-09T00:00:00.000Z,PECO
1,-75.166112,39.941477,215645,AAPT,25.0,1997-06-10T00:00:00.000Z,Streets
2,-75.163483,39.943068,215926,WP,NaN,1997-06-04T00:00:00.000Z,PECO
3,-75.167727,39.944528,214515,WP,25.0,1997-06-09T00:00:00.000Z,Streets
4,-75.171863,39.941486,214380,WP,25.0,1997-06-09T00:00:00.000Z,Streets


time: 965 ms


In [0]:
df.shape

(150, 7)

time: 2.61 ms


In [0]:
import plotly_express as px
px.set_mapbox_access_token("pk.eyJ1Ijoic2hha2Fzb20iLCJhIjoiY2plMWg1NGFpMXZ5NjJxbjhlM2ttN3AwbiJ9.RtGYHmreKiyBfHuElgYq_w")
px.scatter_mapbox(df, lat="Y", lon="X",  zoom=15)

time: 905 ms


In [0]:
df["geom"] =  df["Y"].map(str)  + ',' + df['X'].map(str)
df.head()

,X,Y,POLE_NUM,TYPE,HEIGHT,POLE_DATE,OWNER,geom
0,-75.170097,39.942766,214423,WP,NaN,1997-06-09T00:00:00.000Z,PECO,"39.9427660880249,-75.17009743393821"
1,-75.166112,39.941477,215645,AAPT,25.0,1997-06-10T00:00:00.000Z,Streets,"39.941477314134396,-75.166112027818"
2,-75.163483,39.943068,215926,WP,NaN,1997-06-04T00:00:00.000Z,PECO,"39.9430681055253,-75.1634826347411"
3,-75.167727,39.944528,214515,WP,25.0,1997-06-09T00:00:00.000Z,Streets,"39.9445279530865,-75.16772651013079"
4,-75.171863,39.941486,214380,WP,25.0,1997-06-09T00:00:00.000Z,Streets,"39.9414861490123,-75.17186304554191"


time: 48.8 ms


In [0]:
df.geom[0]



'39.9427660880249,-75.17009743393821'

time: 8.51 ms


In [0]:
locator = Nominatim(user_agent="myGeocoder", timeout=10)
rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.001)
rgeocode

time: 7.06 ms


In [0]:
tqdm.pandas()
df['address'] = df['geom'].progress_apply(rgeocode)

100%|██████████| 150/150 [01:44<00:00,  1.43it/s]

time: 1min 44s


In [0]:
df.head()


,X,Y,POLE_NUM,TYPE,HEIGHT,POLE_DATE,OWNER,geom,address
0,-75.170097,39.942766,214423,WP,NaN,1997-06-09T00:00:00.000Z,PECO,"39.9427660880249,-75.17009743393821","(723, South Chadwick Street, Hawthorne, South ..."
1,-75.166112,39.941477,215645,AAPT,25.0,1997-06-10T00:00:00.000Z,Streets,"39.941477314134396,-75.166112027818","(Tindley Temple United Methodist Church, 750, ..."
2,-75.163483,39.943068,215926,WP,NaN,1997-06-04T00:00:00.000Z,PECO,"39.9430681055253,-75.1634826347411","(South 13th Street, Hawthorne, South Philadelp..."
3,-75.167727,39.944528,214515,WP,25.0,1997-06-09T00:00:00.000Z,Streets,"39.9445279530865,-75.16772651013079","(1538, Naudain Street, Rittenhouse Square, Phi..."
4,-75.171863,39.941486,214380,WP,25.0,1997-06-09T00:00:00.000Z,Streets,"39.9414861490123,-75.17186304554191","(Marion Anderson Community Center, Catharine S..."


time: 28 ms


In [0]:
df["address"][0]

Location(723, South Chadwick Street, Hawthorne, South Philadelphia, Philadelphia, Philadelphia County, Pennsylvania, 19146, United States, (39.9427343888889, -75.1700092777778, 0.0))

time: 3.78 ms
